In [2]:
import numpy as np
import ot
import torch
from math import sqrt
import timeit

from aux_functions import *
from TheGAN import LevyGAN
import configs_folder.configs as configs

In [10]:
actual_bsz = 65536
_s_dim = 16
total_len = actual_bsz * _s_dim

start_time = timeit.default_timer()
for i in range(2):
    result = torch.randperm(actual_bsz * _s_dim)[:actual_bsz]

elapsed = timeit.default_timer() - start_time
print(elapsed)

2.1650694419986394


In [11]:
start_time = timeit.default_timer()
for i in range(2):
    idx = 0
    result = []
    for j in range(actual_bsz):
        result.append(idx)
        idx = (idx + actual_bsz + 1) % total_len

elapsed = timeit.default_timer() - start_time
print(elapsed)

1.098054696998588


In [3]:
config = {
    'w dim': 4,
    'noise size': 62,
    'which generator': 4,
    'which discriminator': 4,
    'generator symmetry mode': 'Hsym',
    'leakyReLU slope': 0.2,
    'test bsz': 16384,
    'unfixed test bsz': 16384,
    'joint wass dist bsz': 1,
    'num tests for 2d': 8,
    'W fixed whole': [1.0, -0.5, -1.2, -0.3, 0.7, 0.2, -0.9, 0.1, 1.7],
    'do timeing': True
}

training_config = configs.training_config

levG = LevyGAN(config)
levG.load_dicts_unstructured('generator_num12_lrG_0.000010_lrD_0.000100min_chen.pt', 'discriminator_num12_lrG_0.000010_lrD_0.000100min_chen.pt')
print("blub")
levG.do_tests()
print(levG.make_report())

blub
Z FOR REPORT TIME: 0.028695765000520623
RUNNING netG FOR REPORT TIME: 0.3125595519995841
UNFIXED PART OF REPORT TIME: 1.0519997886149213e-06


/home/andy/anaconda3/envs/GAN/lib/python3.10/site-packages/ot/lp/solver_1d.py:41: UserWarning: The use of `x.T` on tensors of dimension other than 2 to reverse their shape is deprecated and it will throw an error in a future release. Consider `x.mT` to transpose batches of matricesor `x.permute(*torch.arange(x.ndim - 1, -1, -1))` to reverse the dimensions of a tensor. (Triggered internally at  /opt/conda/conda-bld/pytorch_1659484746364/work/aten/src/ATen/native/TensorShape.cpp:2981.)
  cws = cws.T.contiguous()


CHEN ERRORS TIME: 0.4485517089997302
FIXED ERRORS TIME: 0.6242633250003564
ST DEV ERRORS TIME: 0.007530912000220269
JOINT WASS ERRORS TIME: 1.225999767484609e-06
discr grad norm: 0.00000, discr loss: 0.00000, st dev err: 0.13991
errs: ['0.0457', '0.0295', '0.0432', '0.0745', '0.0975', '0.0540'], chen errs: ['0.3063', '0.2958', '0.3047', '0.2814', '0.2833', '0.2917']


In [4]:
levG.joint_wass_dist_bsz = 16384
levG.do_tests(comp_joint_err=True)
print(levG.make_report())

Z FOR REPORT TIME: 5.398592878999807
RUNNING netG FOR REPORT TIME: 0.36162599500039505
UNFIXED PART OF REPORT TIME: 1.6919993868214078e-06
CHEN ERRORS TIME: 0.5044026489995304
FIXED ERRORS TIME: 0.7854472899998655
ST DEV ERRORS TIME: 0.012961273000655638
JOINT WASS ERRORS TIME: 131.71611658799975
discr grad norm: 0.00000, discr loss: 0.00000, joint err: 0.32943, st dev err: 0.14001
errs: ['0.0480', '0.0293', '0.0425', '0.0763', '0.0978', '0.0522'], chen errs: ['0.3035', '0.3000', '0.3103', '0.2845', '0.2822', '0.2952']


In [4]:
print(levG.test_results['chen errors'])

[0.539863544106743, 0.5614216322454428, 0.25632986824445086, 0.5400680892897032, 0.2564067082281859, 0.48637749907715033]


In [3]:
levG.classic_train(training_config)

512


/home/andy/anaconda3/envs/GAN/lib/python3.10/site-packages/ot/lp/solver_1d.py:41: UserWarning: The use of `x.T` on tensors of dimension other than 2 to reverse their shape is deprecated and it will throw an error in a future release. Consider `x.mT` to transpose batches of matricesor `x.permute(*torch.arange(x.ndim - 1, -1, -1))` to reverse the dimensions of a tensor. (Triggered internally at  /opt/conda/conda-bld/pytorch_1659484746364/work/aten/src/ATen/native/TensorShape.cpp:2981.)
  cws = cws.T.contiguous()


ep: 0/30, itr: 0, discr grad norm: 0.08651, discr loss: -8.08579, joint err: 0.54730, st dev err: 0.14189
errs: ['0.0335', '0.0654', '0.0436', '0.0316', '0.1766', '0.0632'], chen errs: ['0.5116', '0.5618', '0.2511', '0.5165', '0.3604', '0.2753']
Saved parameters (fixed error)
Saved parameters (chen errors)



KeyboardInterrupt



In [2]:
beta2 = 0.98
for beta1 in [0.05,0.5]:
    for msch in ['_min_sum', 'min_chen']:
        dsc = f"beta1_{beta1:.2f}_beta2_{beta2:.3f}{msch}"
        levG.load_dicts(serial_num_to_load=6, descriptor=dsc)
        print("loaded dicts")
        levG.do_tests(comp_joint_err=False, comp_grad_norm=False)
        print(f"{dsc}: chen errors: {make_pretty(levG.test_results['chen errors'])}")


blub
loaded dicts
65536
torch.Size([1048576, 10])


/home/andy/anaconda3/envs/GAN/lib/python3.10/site-packages/ot/lp/solver_1d.py:41: UserWarning: The use of `x.T` on tensors of dimension other than 2 to reverse their shape is deprecated and it will throw an error in a future release. Consider `x.mT` to transpose batches of matricesor `x.permute(*torch.arange(x.ndim - 1, -1, -1))` to reverse the dimensions of a tensor. (Triggered internally at  /opt/conda/conda-bld/pytorch_1659484746364/work/aten/src/ATen/native/TensorShape.cpp:2981.)
  cws = cws.T.contiguous()


beta1_0.05_beta2_0.980_min_sum: chen errors: ['0.4903', '0.4896', '0.2182', '0.4989', '0.2216', '0.2310']
loaded dicts
65536
torch.Size([1048576, 10])
beta1_0.05_beta2_0.980min_chen: chen errors: ['0.4428', '0.4521', '0.1645', '0.4542', '0.2168', '0.1754']
loaded dicts
65536
torch.Size([1048576, 10])
beta1_0.50_beta2_0.980_min_sum: chen errors: ['0.4950', '0.5012', '0.2236', '0.5027', '0.2285', '0.2199']
loaded dicts
65536
torch.Size([1048576, 10])
beta1_0.50_beta2_0.980min_chen: chen errors: ['0.4701', '0.4684', '0.1606', '0.4622', '0.2317', '0.1812']


beta1_0.05_beta2_0.980_min_sum:  joint err: 0.45982, chen errors: ['0.4920', '0.4912', '0.2205', '0.4960', '0.2204', '0.2306']
beta1_0.05_beta2_0.980min_chen:  joint err: 0.38241, chen errors: ['0.4405', '0.4509', '0.1649', '0.4532', '0.2158', '0.1753']
beta1_0.50_beta2_0.980_min_sum:  joint err: 0.48113, chen errors: ['0.4937', '0.5019', '0.2218', '0.4981', '0.2285', '0.2204']
beta1_0.50_beta2_0.980min_chen:  joint err: 0.38037, chen errors: ['0.4708', '0.4636', '0.1610', '0.4650', '0.2306', '0.1820']

In [2]:
print(select_pruning_indices(8,20))

[0, 1, 2, 23, 24, 25, 46, 47, 48, 69, 70, 71, 92, 93, 94, 115, 116, 117, 138, 139, 140]
